# Random Forest

# Kutubxonalar 

In [5]:
import pandas as pd
import numpy as np
import seaborn as snb
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold

In [6]:
# Data yuklash
df=pd.read_csv("Carseats.csv")

In [7]:
#Data xaqida umumiy malumot
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    object 
 6   ShelveLoc    399 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    object 
 9   Urban        400 non-null    object 
 10  US           397 non-null    object 
dtypes: float64(1), int64(5), object(5)
memory usage: 34.5+ KB


In [8]:
#Null qiymatlar 
df.isnull().sum()

Sales          0
CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      1
Age            0
Education      0
Urban          0
US             3
dtype: int64

In [9]:
#Umumiy barcha null qiymatlar uchun tozalash
for col in df.columns:
    if df[col].isnull().sum()>0:
        if df[col].dtype=='object':
            df[col].fillna(df[col].mode()[0], inplace=True)
        else:
            df[col].fillna(df[col].mean(), inplace=True)

/var/folders/1p/wjjmstx57b73dwzmvlpmj7jr0000gn/T/ipykernel_42079/3774126143.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [10]:
df.isnull().sum()

Sales          0
CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      0
Age            0
Education      0
Urban          0
US             0
dtype: int64

In [11]:
# objectni int yoki floatga otkazish
categorical_col=df.select_dtypes(include=['object', 'category']).columns
label_encoder=LabelEncoder()
for col in categorical_col:
    df[col]=label_encoder.fit_transform(df[col])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    int64  
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    int64  
 10  US           400 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 34.5 KB


In [13]:
# Datani input/outputga ajratish
x=df.drop('US',axis=1)

In [14]:
x.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban
0,9.50,138,73,11,276,20,2,42,7,1
1,11.22,111,48,16,260,85,3,65,0,1
2,10.06,113,35,10,269,82,4,59,2,1
3,7.40,117,100,4,466,98,4,55,4,1
4,4.15,141,64,3,340,28,2,38,3,1


In [16]:
y=df['US']

# Train_test_split

In [17]:
x_train,x_temp,y_train,y_temp=train_test_split(x,y,test_size=0.2, random_state=42)


In [18]:
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,test_size=0.5,random_state=42)

In [19]:
len(x_train)

320

In [20]:
len(x_val)

40

# model aniqlash

In [22]:
model=RandomForestRegressor()

In [23]:
model.fit(x_train,y_train)

RandomForestRegressor()

# Prediction 

In [24]:
y_pred=model.predict(x_test)

In [25]:
y_pred

array([0.97, 0.97, 0.27, 0.94, 0.4 , 1.  , 0.07, 1.  , 0.19, 0.18, 0.9 ,
       0.31, 0.1 , 0.12, 1.  , 1.  , 0.04, 0.82, 0.29, 1.  , 0.84, 0.07,
       0.33, 0.11, 0.98, 0.69, 1.  , 1.  , 0.88, 0.02, 1.  , 0.94, 1.  ,
       1.  , 0.81, 1.  , 1.  , 0.79, 0.5 , 0.02])

# Tekshirish

In [26]:
mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)

In [27]:
print(mse)
print(r2)

0.07343249999999998
0.6652649572649573


# Scaling 

In [28]:
scaler_x=StandardScaler()
scaler_y=StandardScaler()

In [29]:
x_train_scaled=scaler_x.fit_transform(x_train)
x_test_scaled=scaler_x.transform(x_test)

In [30]:
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [31]:
model=RandomForestRegressor()

In [32]:
model.fit(x_train_scaled,y_train_scaled)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [33]:
y_pred_scaled=model.predict(x_test_scaled)

In [34]:
mse=mean_squared_error(y_test_scaled,y_pred_scaled)
r2=r2_score(y_test_scaled, y_pred_scaled)

In [35]:
print(mse)
print(r2)

0.3188817312955243
0.662849002849003
